In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
PAD_IDX = 0
UNK_IDX = 1
BATCH_SIZE = 32

In [12]:
data_train = pd.read_json('/home/anumeha/Documents/ACL-BioNLP-2019-Shared-Task/NLI/dataset/mli_train_v1.json',lines = True)
data_val = pd.read_json('/home/anumeha/Documents/ACL-BioNLP-2019-Shared-Task/NLI/dataset/mli_dev_v1.json',lines = True)

In [13]:
data_train

,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,entailment,23eb94b8-66c7-11e7-a8dc-f45c89b91419,Labs were notable for Cr 1.7 (baseline 0.5 per...,( Labs ( ( were ( notable ( for ( ( ( ( Cr 1.7...,(ROOT (S (NP (NNPS Labs)) (VP (VBD were) (ADJP...,Patient has elevated Cr,( Patient ( has ( elevated Cr ) ) ),(ROOT (S (NP (NN Patient)) (VP (VBZ has) (NP (...
1,contradiction,23eb979c-66c7-11e7-b76c-f45c89b91419,Labs were notable for Cr 1.7 (baseline 0.5 per...,( Labs ( ( were ( notable ( for ( ( ( ( Cr 1.7...,(ROOT (S (NP (NNPS Labs)) (VP (VBD were) (ADJP...,Patient has normal Cr,( Patient ( has ( normal Cr ) ) ),(ROOT (S (NP (NN Patient)) (VP (VBZ has) (NP (...
2,neutral,23eb9986-66c7-11e7-9ef9-f45c89b91419,Labs were notable for Cr 1.7 (baseline 0.5 per...,( Labs ( ( were ( notable ( for ( ( ( ( Cr 1.7...,(ROOT (S (NP (NNPS Labs)) (VP (VBD were) (ADJP...,Patient has elevated BUN,( Patient ( has ( elevated BUN ) ) ),(ROOT (S (NP (NN Patient)) (VP (VBZ has) (NP (...
3,entailment,23eb9ba2-66c7-11e7-9ac1-f45c89b91419,Nystagmus and twiching of R arm was noted.,( ( ( Nystagmus and ) ( twiching ( of ( R arm ...,(ROOT (S (NP (NP (NNP Nystagmus)) (CC and) (NP...,The patient had abnormal neuro exam.,( ( The patient ) ( ( had ( abnormal ( neuro e...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBD h...
4,contradiction,23eb9d64-66c7-11e7-a16b-f45c89b91419,Nystagmus and twiching of R arm was noted.,( ( ( Nystagmus and ) ( twiching ( of ( R arm ...,(ROOT (S (NP (NP (NNP Nystagmus)) (CC and) (NP...,The patient has a normal neuro exam.,( ( The patient ) ( ( has ( a ( normal ( neuro...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBZ h...
5,neutral,23eb9f12-66c7-11e7-8d5d-f45c89b91419,Nystagmus and twiching of R arm was noted.,( ( ( Nystagmus and ) ( twiching ( of ( R arm ...,(ROOT (S (NP (NP (NNP Nystagmus)) (CC and) (NP...,The patient has an acute stroke.,( ( The patient ) ( ( has ( an ( acute stroke ...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBZ h...
6,entailment,23eba108-66c7-11e7-a551-f45c89b91419,The patient was seen by his primary care physi...,( ( The patient ) ( ( was ( ( seen ( by ( his ...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBD w...,The patient has symptoms of a CHF exacerbation.,( ( The patient ) ( ( has ( symptoms ( of ( a ...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBZ h...
7,contradiction,23eba310-66c7-11e7-8840-f45c89b91419,The patient was seen by his primary care physi...,( ( The patient ) ( ( was ( ( seen ( by ( his ...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBD w...,The patient has no symptoms.,( ( The patient ) ( ( has ( no symptoms ) ) . ) ),(ROOT (S (NP (DT The) (NN patient)) (VP (VBZ h...
8,neutral,23eba502-66c7-11e7-a947-f45c89b91419,The patient was seen by his primary care physi...,( ( The patient ) ( ( was ( ( seen ( by ( his ...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBD w...,The patient has has coronary artery disease.,( ( The patient ) ( ( has ( has ( coronary ( a...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBZ h...
9,entailment,23eba750-66c7-11e7-a9bc-f45c89b91419,At [**Hospital 1456**] Hospital the patient wa...,( ( At ( -LSB- ( ** ( ( Hospital 1456 ) ( ** (...,(ROOT (SBAR (IN At) (S (-LRB- -LSB-) (X (SYM *...,The patient is being treated with medications...,( ( The patient ) ( ( is ( being ( treated ( w...,(ROOT (S (NP (DT The) (NN patient)) (VP (VBZ i...


In [14]:
data_train = data_train.drop(['pairID','sentence1_binary_parse','sentence1_parse','sentence2_binary_parse','sentence2_parse'],axis = 1)
data_val = data_val.drop(['pairID','sentence1_binary_parse','sentence1_parse','sentence2_binary_parse','sentence2_parse'],axis = 1)

In [15]:
def tokenize(df):
    df['sentence1'] = df['sentence1'].apply(lambda x: [a.lower() for a in x.split(' ')])
    df['sentence2'] = df['sentence2'].apply(lambda x : [a.lower() for a in x.split(' ')])
    
    return df

In [16]:
df_train = tokenize(data_train)
df_val = tokenize(data_val)

In [17]:
df_train.head()

,gold_label,sentence1,sentence2
0,entailment,"[labs, were, notable, for, cr, 1.7, (baseline,...","[, patient, has, elevated, cr]"
1,contradiction,"[labs, were, notable, for, cr, 1.7, (baseline,...","[, patient, has, normal, cr]"
2,neutral,"[labs, were, notable, for, cr, 1.7, (baseline,...","[, patient, has, elevated, bun]"
3,entailment,"[nystagmus, and, twiching, of, r, arm, was, no...","[, the, patient, had, abnormal, neuro, exam.]"
4,contradiction,"[nystagmus, and, twiching, of, r, arm, was, no...","[, the, patient, has, a, normal, neuro, exam.]"


In [18]:
words_to_load = 100000
with open('glove.6B.300d.txt') as f:
    loaded_embeddings = np.zeros((words_to_load+2, 300))
    words = {}
    idx2words = {}
    ordered_words = ['<pad>','<unk>']
    i = 2
    for line in f:
        if i-2 >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings[i, :] = np.asarray(s[1:])
        words[s[0]] = i
        idx2words[i] = s[0]
        ordered_words.append(s[0])
        i = i+1

max(words.values())



100001

In [19]:
words['<pad>'] = 0
words['<unk>'] = 1
idx2words[0] = "<pad>"
idx2words[1] = "<unk>"

In [20]:
def apply_idz(x):
    temp = []
    for a in x:
        if a in ordered_words:
            temp.append(words[a])
        else:
            temp.append(1)
    return temp

In [21]:
def idize(df):
    df['sentence1_idz'] = df["sentence1"].apply(apply_idz)
    df['sentence2_idz'] = df["sentence2"].apply(apply_idz)
    return df


In [22]:
train_train = idize(df_train)
val_val = idize(df_val)

In [23]:
unk_vec = np.random.normal(size=300)
pad_vec = np.random.normal(size=300)

In [24]:
loaded_embeddings[0,:] = pad_vec
loaded_embeddings[1,:] = unk_vec

In [25]:
loaded_embeddings

array([[-0.44151706,  0.70952845, -1.26538153, ...,  0.85752989,
        -0.9351183 , -0.80206106],
       [ 1.30215393,  2.59685577, -1.60680032, ..., -0.31194044,
         0.05851155, -0.41334619],
       [ 0.04656   ,  0.21318   , -0.0074364 , ...,  0.0090611 ,
        -0.20989   ,  0.053913  ],
       ...,
       [ 0.55336   , -0.14248   ,  0.46174   , ..., -0.65683   ,
         0.096285  ,  0.48947   ],
       [-0.2227    ,  0.37756   , -0.22264   , ...,  0.0069883 ,
         0.11574   , -0.063641  ],
       [ 0.44794   , -0.75508   ,  0.52402   , ..., -0.013342  ,
         0.50648   , -0.64096   ]])

In [26]:
def encode_target(train_train):
    train_train['gold_label'][train_train['gold_label']=='neutral']=0
    train_train['gold_label'][train_train['gold_label']=='entailment']=1
    train_train['gold_label'][train_train['gold_label']=='contradiction']=2
    return train_train
train_train = encode_target(train_train)
val_val = encode_target(val_val)

In [27]:
train_train.head()

,gold_label,sentence1,sentence2,sentence1_idz,sentence2_idz
0,1,"[labs, were, notable, for, cr, 1.7, (baseline,...","[, patient, has, elevated, cr]","[10791, 37, 2785, 12, 17485, 6264, 1, 6322, 53...","[1, 3224, 33, 7795, 17485]"
1,2,"[labs, were, notable, for, cr, 1.7, (baseline,...","[, patient, has, normal, cr]","[10791, 37, 2785, 12, 17485, 6264, 1, 6322, 53...","[1, 3224, 33, 1975, 17485]"
2,0,"[labs, were, notable, for, cr, 1.7, (baseline,...","[, patient, has, elevated, bun]","[10791, 37, 2785, 12, 17485, 6264, 1, 6322, 53...","[1, 3224, 33, 7795, 28256]"
3,1,"[nystagmus, and, twiching, of, r, arm, was, no...","[, the, patient, had, abnormal, neuro, exam.]","[1, 7, 1, 5, 1913, 2649, 17, 1]","[1, 2, 3224, 42, 15820, 79229, 1]"
4,2,"[nystagmus, and, twiching, of, r, arm, was, no...","[, the, patient, has, a, normal, neuro, exam.]","[1, 7, 1, 5, 1913, 2649, 17, 1]","[1, 2, 3224, 33, 9, 1975, 79229, 1]"


In [28]:
class MednliDataset(Dataset):
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        s1 = self.df.iloc[idx]['sentence1_idz']
        s2 = self.df.iloc[idx]['sentence2_idz']
        tar = self.df.iloc[idx]['gold_label']
        len1 = len(s1)
        len2 = len(s2)
        return [s1,len1,s2,len2,tar]

In [29]:
def max_len(x):
    return len(x)

In [30]:
l1 = train_train['sentence1_idz'].apply(max_len)
l2 = train_train['sentence2_idz'].apply(max_len)
# MAX_LEN = max(l1)
MAX_LEN = 25

In [31]:
def vocab_collate_func(batch):
    data_list_s1 = []
    data_list_s2 = []
    label_list = []
    length_list_s1 = []
    length_list_s2 = []

    for datum in batch:
        label_list.append(datum[4])
        length_list_s1.append(datum[1])
        length_list_s2.append(datum[3])
    # padding
    for datum in batch:
        if datum[1]>MAX_LEN:
            padded_vec_s1 = np.array(datum[0])[:MAX_LEN]
        else:
            padded_vec_s1 = np.pad(np.array(datum[0]),
                                pad_width=((0,MAX_LEN - datum[1])),
                                mode="constant", constant_values=0)
        if datum[3]>MAX_LEN:
            padded_vec_s2 = np.array(datum[2])[:MAX_LEN]
        else:
            padded_vec_s2 = np.pad(np.array(datum[2]),
                                pad_width=((0,MAX_LEN - datum[3])),
                                mode="constant", constant_values=0)
        data_list_s1.append(padded_vec_s1)
        data_list_s2.append(padded_vec_s2)
        
    return [torch.from_numpy(np.array(data_list_s1)), torch.LongTensor(length_list_s1), torch.from_numpy(np.array(data_list_s2)), torch.LongTensor(length_list_s2), torch.LongTensor(label_list)]


In [32]:
train_dataset = MednliDataset(train_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)

val_dataset = MednliDataset(val_val)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)

In [33]:
class RNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes, vocab_size):
        super(RNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.embedding.from_pretrained(torch.from_numpy(loaded_embeddings), freeze = True)


        
        self.rnn = nn.GRU(emb_size, hidden_size, num_layers, batch_first = True)
        self.linear1 = nn.Linear(2*hidden_size, 500)
        self.linear2 = nn.Linear(500,num_classes)

    def init_hidden(self, batch_size):
        hidden = torch.randn(self.num_layers, batch_size, self.hidden_size)
        return hidden

    def forward(self, data_s1, length1, data_s2, length2):  
        batch_size = data_s1.size(0)
        self.hidden1 = self.init_hidden(batch_size)
        self.hidden2 = self.init_hidden(batch_size)

        embed1 = self.embedding(data_s1)
        embed2 = self.embedding(data_s2)
        
#         print(embed1.size())
        rnn_out1_, hidden1 = self.rnn(embed1, self.hidden1)
        rnn_out2_, hidden2 = self.rnn(embed2, self.hidden2)
#         print(self.hidden1.size())

        rnn_out1 = torch.sum(hidden1, dim=0)
        rnn_out2 = torch.sum(hidden2, dim=0)
#         print(rnn_out1.size())
        combined_out = torch.cat([rnn_out1, rnn_out2], dim=1)
#         print(combined_out.size())
        logits = F.relu(self.linear1(combined_out))
        res = self.linear2(logits)
        
        return res

In [34]:
dataloaders = [train_loader, val_loader]

In [35]:
def training(model,criterion, optimizer, name, num_epochs):
    best_loss = np.inf
    best_acc = 0
    acc_hist = {'train':[],'validate':[]}
    for i in range(num_epochs):
        for enu,phase in enumerate(['train', 'validate']):
            running_loss = 0
            running_total = 0
            correct = 0
            total = 0
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)
            for (data1, length1, data2, length2, labels) in dataloaders[enu]:
                data_batch1, len_batch1, data_batch2, len_batch2, label_batch = data1, length1, data2, length2, labels
                

                optimizer.zero_grad()
                outputs = model(data_batch1, len_batch1, data_batch2, len_batch2)
                loss = criterion(outputs, label_batch)
                if phase=='train':
                    loss.backward()
                    optimizer.step()
                N = labels.size(0)
                
                outputs = F.softmax(outputs, dim=1)
                predicted = outputs.max(1, keepdim=True)[1]
#                 print(type(predicted))
                total += labels.size(0)
                correct += predicted.eq(label_batch.view_as(predicted)).sum().item()
               
            accuracy = 100 * correct / total
            acc_hist[phase].append(accuracy)
            print('Epoch: {}, Phase: {}, accuracy: {:.4f}'\
                      .format(i,phase, accuracy))
        if phase == 'validate' and best_acc < accuracy:
            best_acc = accuracy
            torch.save(model,name)
    print(' Best Accuracy: {:4f}'.format(best_acc))
    return model, acc_hist

In [37]:
model = RNN(300,500,1,3,words_to_load+2)
criterion = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

m_save, loss_hists,acc_hist = training(model,criterion,optimizer,"model_rnn1",30)

Epoch: 0, Phase: train, accuracy: 32.7368
Epoch: 0, Phase: validate, accuracy: 33.3333
Epoch: 1, Phase: train, accuracy: 33.3244
Epoch: 1, Phase: validate, accuracy: 33.3333
Epoch: 2, Phase: train, accuracy: 32.2026
Epoch: 2, Phase: validate, accuracy: 33.8351
Epoch: 3, Phase: train, accuracy: 42.5659
Epoch: 3, Phase: validate, accuracy: 57.2043
Epoch: 4, Phase: train, accuracy: 61.2892
Epoch: 4, Phase: validate, accuracy: 64.8746
Epoch: 5, Phase: train, accuracy: 67.9754
Epoch: 5, Phase: validate, accuracy: 66.0215
Epoch: 6, Phase: train, accuracy: 71.9017
Epoch: 6, Phase: validate, accuracy: 66.5233
Epoch: 7, Phase: train, accuracy: 76.0506
Epoch: 7, Phase: validate, accuracy: 66.3799
Epoch: 8, Phase: train, accuracy: 79.0954
Epoch: 8, Phase: validate, accuracy: 66.6667
Epoch: 9, Phase: train, accuracy: 82.4519
Epoch: 9, Phase: validate, accuracy: 66.1649
Epoch: 10, Phase: train, accuracy: 85.3187
Epoch: 10, Phase: validate, accuracy: 66.4516
Epoch: 11, Phase: train, accuracy: 87.749

KeyboardInterrupt: 